***Libraries***

In [1]:
import transformers
import datasets
from datasets import Dataset,load_dataset,DatasetDict
import peft
import bitsandbytes
import accelerate
import evaluate
from evaluate import evaluator
import seqeval
from transformers import pipeline, AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments,DataCollatorWithPadding
from transformers import AutoModelForQuestionAnswering,BitsAndBytesConfig,AutoModelForTokenClassification,default_data_collator
from peft import LoraConfig,get_peft_model,TaskType,PeftModel,prepare_model_for_kbit_training
import numpy as np
import pandas as pd 
import torch
import os
from tqdm import tqdm
import scipy.stats
metric_accuracy = evaluate.load('accuracy')
metric_f1 = evaluate.load("f1")
metric_seqeval = evaluate.load("seqeval") 
metric_squad = evaluate.load("squad")
import warnings
warnings.filterwarnings(action = 'ignore')

C:\Users\Hasan Yusufzada\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The 8-bit optimizer is not available on your device, only available on CUDA for now.


In [2]:
from transformers import DataCollatorForTokenClassification

In [3]:
import random

In [4]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)

***BERT-PEFT-LORA***

In [5]:
model_name = "bert-base-uncased"
dataset_name = "conll2003"
low_resource_samples = 512

In [6]:
def prepare_ner_data():
    
    dataset = load_dataset(dataset_name)
     
    train_dataset_small = dataset["train"].select(range(low_resource_samples))
    
   
    label_names = dataset["train"].features["ner_tags"].feature.names
    
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
   
    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples["tokens"], 
            truncation=True, 
            is_split_into_words=True,
            max_length=512,
            padding=False
        )
        
        labels = []
        for i, label in enumerate(examples["ner_tags"]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(-100)
                previous_word_idx = word_idx
            labels.append(label_ids)
        
        tokenized_inputs["labels"] = labels
        return tokenized_inputs
    
    
    tokenized_train = train_dataset_small.map(tokenize_and_align_labels, batched=True)
    tokenized_eval = dataset["validation"].map(tokenize_and_align_labels, batched=True)
    

    data_collator = DataCollatorForTokenClassification(
        tokenizer=tokenizer, 
        padding=True
    )
    
    return tokenized_train, tokenized_eval, data_collator, label_names, tokenizer


def compute_ner_metrics(eval_pred, label_names):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)
    
    true_predictions = []
    true_labels = []
    
    for i in range(len(predictions)):
        prediction = predictions[i]
        label = labels[i]
        valid_preds = [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        valid_labels = [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        
        true_predictions.append(valid_preds)
        true_labels.append(valid_labels)
    
    metric = evaluate.load("seqeval")
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [7]:
tokenized_train, tokenized_eval, data_collator, label_names, tokenizer = prepare_ner_data()


model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_names),
    id2label={i: label for i, label in enumerate(label_names)},
    label2id={label: i for i, label in enumerate(label_names)}
)


lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "key", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.TOKEN_CLS
)


model = get_peft_model(model, lora_config)


model.print_trainable_parameters()


training_args = TrainingArguments(
    output_dir="./ner_lora",
    eval_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs_ner_lora",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    report_to="none",
    seed=42
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=lambda eval_pred: compute_ner_metrics(eval_pred, label_names)
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 449,289 || all params: 109,347,858 || trainable%: 0.4109


In [8]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.641100,0.572603,0.478261,0.042578,0.078195,0.838889
2,0.312300,0.339465,0.509836,0.532144,0.520751,0.913925
3,0.243600,0.271742,0.608805,0.653989,0.630588,0.930357
4,0.180900,0.248923,0.632164,0.679401,0.654932,0.937094
5,0.196100,0.242319,0.634589,0.689162,0.660750,0.938281


TrainOutput(global_step=320, training_loss=0.4273364905267954, metrics={'train_runtime': 2533.0159, 'train_samples_per_second': 1.011, 'train_steps_per_second': 0.126, 'total_flos': 52493551377120.0, 'train_loss': 0.4273364905267954, 'epoch': 5.0})

In [9]:
results = trainer.evaluate()
print("LoRA-NER", results)

trainer.save_model("./NER_LoRa_Model")

LoRA-NER {'eval_loss': 0.24231912195682526, 'eval_precision': 0.6345885634588564, 'eval_recall': 0.6891618983507236, 'eval_f1': 0.6607503025413474, 'eval_accuracy': 0.9382812195786768, 'eval_runtime': 374.211, 'eval_samples_per_second': 8.685, 'eval_steps_per_second': 0.545, 'epoch': 5.0}
